# Blue Book for Bulldozers

In this example, we will create a model to solve the [Blue Book for Bulldozers](https://www.kaggle.com/c/bluebook-for-bulldozers) Kaggle competition. For this competition, we should predict the sale price of bulldozers sold at auctions.

First, we observe that this is a regression task, since we are predicting a real-valued number. For this tutorial, the dataset is one CSV file, containing more than 20.000 examples, each one having 51 features (if you exclude the `SalesID` and the `SalesPrice` attributes).

## Install

First things first, we need to install the necessary dependencies. We can do that by either running `!pip install --user <package_name>` or including everything in a `requirements.txt` file and running `!pip install --user -r requirements.txt`. We have put the dependencies in a `requirements.txt` file already, so we will use the second method.

> NOTE: Do not forget to use the `--user` argument. It is necessary if you want to use Kale to transform this notebook into a Kubeflow pipeline

In [ ]:
!pip3 install --user -r requirements.txt

## Imports

In this section we import the packages we need for this example. When working in a Jupyter Notebook, it is a good habit to gather your imports in a single place.

In [ ]:
import numpy as np
import pandas as pd

from kubeflow import katib
from kale import ml as kale_ml
from kale.common import serveutils
from autosklearn import metrics
from fastai.tabular import core

## Data Loading

In this section we load the dataset and do some light processing. Specifically, we need to turn every categorical feature into a number. First, let's load the data.

In [2]:
df = pd.read_csv("data/train.csv", low_memory=False)
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1597691,10000.0,1204623,4600,132,18.0,1979,NaN,NaN,4/21/1994 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1363686,10000.0,1149235,7267,132,8.0,1978,NaN,NaN,3/27/2002 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1767435,64000.0,1164512,28919,132,1.0,2006,NaN,NaN,3/15/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
3,1639287,24500.0,1457804,1894,132,15.0,2001,NaN,NaN,11/4/2010 0:00,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN
4,2277587,21750.0,572911,2758,136,20.0,1998,5155.0,Medium,4/16/2008 0:00,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN


## Data Processing

Let's go through the transformations we need to do to our data:

1. Extract the target variable (`SalePrice`)
2. Encode the ordinal variables
3. Unfold the dates to engineer more features, and
4. Split the dataset into `train` and `valid` sets

First, let's keep our target in a variable:

In [3]:
target_var = 'SalePrice'

The next step is to encode the ordinal variables. Why do we treat the `ProductSize` variable differently? It is because the order here does matter. So if we want to assign a number to every value that this variable can take, we need to account for that. Thus, `Large` can take the value `1`, `Large / Medium` the value `2`, and so on.

In [4]:
# ordinal data
sizes = 'Large', 'Large / Medium', 'Medium', 'Small', 'Mini', 'Compact'

df['ProductSize'] = df['ProductSize'].astype('category')
df['ProductSize'].cat.set_categories(sizes, ordered=True, inplace=True)

Next, we need to unfold the `date` feature to extract more information out of it. For example, it makes sense to know if it's Christmas, or a Weekend, if it's Summer or Winter, or even which day of the week it is.

To achieve this, we will use a handy function provided by the [fastai](https://docs.fast.ai/) library: `add_datepart`.

In [5]:
# expand dates
df = core.add_datepart(df, 'saledate')

Now, we are ready to split our dataset into `train` and `test` sets. Since, we are trying to predict the sale price of bulldozers in some auction, we should be extra careful about how we split our data. We want the validation set to be later in time than the training set. We shouldn't allow future values to leak into our training set.

In [6]:
# create splits
condition = (df.saleYear<2011) | (df.saleMonth<10)
train_idx = np.where(condition)[0]
valid_idx = np.where(~condition)[0]
splits = (list(train_idx), list(valid_idx))

# locate continuous and categorigal features
cont, cat = core.cont_cat_split(df, 1, dep_var=target_var)

# preprocess the dataset
df_proc = core.TabularPandas(df, [core.Categorify], cat, cont, y_names=target_var, splits=splits)

Now that we have finished processing our dataset, we are ready to extract our features and labels into `numpy` arrays.

In [7]:
# create feature and target matrices
X_train = df_proc.train.items.drop("SalePrice", axis=1).values
y_train = df_proc.train.items["SalePrice"].values
X_valid = df_proc.valid.items.drop("SalePrice", axis=1).values
y_valid = df_proc.valid.items["SalePrice"].values

Kale provides a useful abstraction to group together our dataset. We just need to provide the `X` and `y` values.

In [8]:
# create Kale Dataset
dataset = kale_ml.Dataset(
    features=X_train,
    targets=y_train,
    features_test=X_valid,
    targets_test=y_valid,
    name="bluebook-bulldozers")

Finally, we are ready to run our AutoML experiment using Kale. We need three things:

* The dataset
* The task (in our case regression)
* The metric we are trying to optimize

Moreover, we can pass a parameter, to let Kale know how many different ML configurations it should try, and a Katib specification, if you want to further optimize the best performing predictor.

In [9]:
# create the Katib configuration
tuner = katib.V1beta1ExperimentSpec(
    objective=katib.V1beta1ObjectiveSpec(
        goal=0.,
        type="maximize"
    ),
    max_trial_count=2,
    parallel_trial_count=1
)

In [11]:
# execute the AutoML experiment
automl = kale_ml.run_automl(dataset,
                            kale_ml.Task.REGRESSION,
                            metric=metrics.mean_squared_log_error,
                            number_of_configurations=4,
                            max_parallel_configurations=2,
                            tuner=tuner)

2021-04-22 11:45:09 Kale marshalling          [INFO]     Saving generic object using Default backend: dataset
2021-04-22 11:45:09 Kale marshalling          [INFO]     Saving generic object using Default backend: task
2021-04-22 11:45:09 Kale marshalling          [INFO]     Saving scikit-learn object using SKLearn backend: metric
2021-04-22 11:45:09 Kale marshalling          [INFO]     Saving generic object using Default backend: number_of_configurations
2021-04-22 11:45:09 Kale marshalling          [INFO]     Saving generic object using Default backend: max_parallel_configurations
2021-04-22 11:45:09 Kale marshalling          [INFO]     Saving generic object using Default backend: tuner
2021-04-22 11:45:09 Kale rokutils:156         [INFO]     Taking a snapshot of the Pod's volumes...
2021-04-22 11:45:09 Kale rokutils:105         [INFO]     Taking a snapshot of pod kubecon-tutorial-0 in namespace kubeflow-user ...
2021-04-22 11:45:09 Kale rokutils:313         [INFO]     Creating Rok buc

2021-04-22 11:45:22 Kale kfputils:175         [INFO]     Submitting new pipeline run 'automl-orchestrate-vu38n-dk9u7' for pipeline 'automl-orchestrate' (version: 'vu38n') ...


2021-04-22 11:45:22 Kale kfputils:182         [INFO]     Successfully submitted pipeline run.
2021-04-22 11:45:22 Kale kfputils:183         [INFO]     Run URL: <host>/pipeline/?ns=kubeflow-user#/runs/details/6ff92eaf-3d9a-4190-9318-d6d29d78100c


You can monitor the experiment by printing a summary of the AutoML task at any point in time.

In [12]:
automl.summary()

AutoML Orchestrator status: Succeeded

4/4 Configuration Runs have started.



Status,Count
Running,0
Succeeded,4
Skipped,0
Failed,0
Error,0


#,KFP Run,Status,Metric (mean_squared_log_error)
1,c7ab64ce-b59a-4193-ad75-098065e59666,Succeeded,-0.467985
2,0600f3e3-855c-411d-8034-6aaf76417b5d,Succeeded,-0.0687647
3,3f2d62bc-2a99-40c6-9237-c61525acc31c,Succeeded,-0.483285
4,ccbab346-776f-46f3-85bf-5c79b03c98f2,Succeeded,-0.257837


# Serve the best-performing model

Now that we have found the best configuration and performed hyperparameter optimization on it, let's serve the best-performing model.

In [ ]:
from kale.common import serveutils

kale_model_artifact_id = <KALE_MODEL_ARTIFACT_ID_PLACEHOLDER>

kfserver = serveutils.serve_model_from_mlmd(kale_model_artifact_id)

In [14]:
kfserver

# Run predictions agains the model

In [15]:
import json
data = {"instances": X_valid[0:3].tolist()}
res = kfserver.predict(json.dumps(data))
print(res)

2021-04-22 13:56:39 Kale serveutils:162       [INFO]     Sending a request to the InferenceService...
2021-04-22 13:56:40 Kale serveutils:163       [INFO]     Getting InferenceService's host...
2021-04-22 13:56:40 Kale serveutils:166       [INFO]     Sending request to InferenceService...
2021-04-22 13:56:40 Kale serveutils:172       [INFO]     Response: {"predictions": [46462.40234375, 19027.392578125, 28755.17578125]} ..... 7578125]}


{'predictions': [46462.40234375, 19027.392578125, 28755.17578125]}


# TensorBoard logs

In [16]:
from kale.ml.api import create_tensorboard_from_mlmd
tensorboard_logs_artifact_id = <TENSORBOARD_LOGS_ARTIFACT_ID_PLACEHOLDER>
tb = create_tensorboard_from_mlmd(tensorboard_logs_artifact_id, name="blue-book-bulldozers")

2021-04-22 13:57:00 Kale api:553              [INFO]     Hydrating PVC 'kubeflow-user/blue-book-bulldozers-pua2d-pvc' from Rok URI: rok:kubeflow-user:tensorboard-logs:katib-trial-sklearn-configuration-lbhxh-automl-orchestrate-rwcg5-workspace-kubecon-tutorial-i6nwkt6jc:2e4b6d3f-97ab-403e-bb5e-5f169f720bc9:/prv-kale/backend/kale/ml/examples/tutorial/logs
2021-04-22 13:57:00 Kale rokutils:481         [INFO]     Unpacking Rok URI: 'rok:kubeflow-user:tensorboard-logs:katib-trial-sklearn-configuration-lbhxh-automl-orchestrate-rwcg5-workspace-kubecon-tutorial-i6nwkt6jc:2e4b6d3f-97ab-403e-bb5e-5f169f720bc9:/prv-kale/backend/kale/ml/examples/tutorial/logs'
2021-04-22 13:57:00 Kale rokutils:485         [INFO]     User kubeflow-user; bucket tensorboard-logs; object katib-trial-sklearn-configuration-lbhxh-automl-orchestrate-rwcg5-workspace-kubecon-tutorial-i6nwkt6jc; version 2e4b6d3f-97ab-403e-bb5e-5f169f720bc9; path /prv-kale/backend/kale/ml/examples/tutorial/logs
2021-04-22 13:57:00 Kale rokutil